<a href="http://www.godgeleerdheid.vu.nl/etcbc" target="_blank"><img align="left" src="images/VU-ETCBC-xsmall.png"/></a>
<a href="http://laf-fabric.readthedocs.org/en/latest/" target="_blank"><img align="right" src="images/laf-fabric-xsmall.png"/></a>
<a href="http://www.persistent-identifier.nl/?identifier=urn%3Anbn%3Anl%3Aui%3A13-048i-71" target="_blank"><img align="right"src="images/etcbc4easy-small.png"/></a>

In [1]:
import sys
import collections

import laf
from laf.fabric import LafFabric
from etcbc.preprocess import prepare
from etcbc.lib import Transcription, monad_set
from etcbc.trees import Tree

fabric = LafFabric()
tr = Transcription()

  0.00s This is LAF-Fabric 4.5.0
API reference: http://laf-fabric.readthedocs.org/en/latest/texts/API-reference.html
Feature doc: http://shebanq-doc.readthedocs.org/en/latest/texts/welcome.html



In [25]:
API = fabric.load('etcbc4', '--', 'dicendi', {
    "xmlids": {"node": False, "edge": False},
    "features": ('''
        oid otype monads
        sp vt lex
    ''','''
        mother
    '''),
    "prepare": prepare,
}, verbose='NORMAL')
exec(fabric.localnames.format(var='fabric'))

  0.00s LOADING API: please wait ... 
  0.00s INFO: USING DATA COMPILED AT: 2014-07-23T09-31-37
  1.72s INFO: DATA LOADED FROM SOURCE etcbc4 AND ANNOX -- FOR TASK dicendi AT 2014-09-15T08-49-22


Construct the trees

In [18]:
tree_types = ('clause', 'word')
(root_type, leaf_type, clause_type) = (tree_types[0], tree_types[-1], 'clause')

tree = Tree(API, otypes=tree_types, 
    clause_type=clause_type,
    ccr_feature=None,
    pt_feature=None,
    pos_feature='sp',
    mother_feature =None,
)
results = tree.relations()
parent = results['eparent']
children = results['echildren']
msg("Ready for processing")

  0.00s LOADING API with EXTRAs: please wait ... 
  0.00s INFO: USING DATA COMPILED AT: 2014-07-23T09-31-37
  0.72s INFO: DATA LOADED FROM SOURCE etcbc4 AND ANNOX -- FOR TASK dicendi AT 2014-09-15T08-14-30
  0.00s Start computing parent and children relations for objects of type clause, word
  1.36s 100000 nodes
  2.85s 200000 nodes
  4.23s 300000 nodes
  5.53s 400000 nodes
  6.77s 500000 nodes
  6.96s 514533 nodes: 426555 have parents and 87978 have children
  7.00s Ready for processing


Investigate clauses: get the lexemes of the one word clauses, if the part of speech of that word is verb.

In [34]:
count_clauses = collections.defaultdict(lambda: 0)
lemor = set()

for n in children:
    nwords = len(children[n])
    if nwords == 2:
        prep = children[n][0]
        word = children[n][1]
        if F.sp.v(prep) == 'prep' and F.sp.v(word) == 'verb':
            lex = F.lex.v(word)
            count_clauses[lex] += 1
            if lex == '>MR': lemor.add(word)

print("{} clauses consisting of just a single verb.\n".format(len(count_clauses)))
for item in sorted(count_clauses.items(), key=lambda x: (-x[1], x[0])):
    print("{}: {} x".format(item[0], item[1]))

312 clauses consisting of just a single verb.

>MR[: 936 x
<FH[: 68 x
QR>=[: 67 x
BW>[: 58 x
MLK[: 53 x
HLK[: 46 x
MWT[: 40 x
R>H[: 40 x
JRC[: 36 x
>KL[: 35 x
JD<[: 32 x
CM<[: 21 x
KLH[: 20 x
CMD[: 18 x
CMR[: 17 x
CXT[: 17 x
CRT[: 16 x
CWB[: 16 x
DBR[: 16 x
JY>[: 16 x
HJH[: 15 x
JCB[: 15 x
K<S[: 15 x
CTH[: 14 x
<BD[: 13 x
XJH[: 13 x
<BR[: 11 x
HRG[: 11 x
JLD[: 11 x
<MD[: 10 x
>BD[: 10 x
BNH[: 10 x
CLX[: 10 x
JC<[: 10 x
<ZR[: 9 x
LQX[: 9 x
QWM[: 9 x
NTN[: 8 x
QDC[: 8 x
BQC[: 7 x
NB>[: 7 x
NYL[: 7 x
XV>[: 7 x
<NH=[: 6 x
CKB[: 6 x
HLL[: 6 x
JR>[: 6 x
JRD[: 6 x
LJN[: 6 x
LXM[: 6 x
NKH[: 6 x
NXM[: 6 x
QR>[: 6 x
R<<[: 6 x
TMM[: 6 x
VM>[: 6 x
XRM[: 6 x
ZKR[: 6 x
<LH[: 5 x
GWR[: 5 x
NPL[: 5 x
QVR[: 5 x
VHR[: 5 x
XLL[: 5 x
XWH[: 5 x
XZH[: 5 x
B<R[: 4 x
BWC[: 4 x
CPV[: 4 x
KHN[: 4 x
KWL[: 4 x
MY>[: 4 x
NF>[: 4 x
RBH[: 4 x
XMM[: 4 x
XSH[: 4 x
XZQ[: 4 x
YBH[: 4 x
<CQ[: 3 x
BJN[: 3 x
BKH[: 3 x
CB<[: 3 x
CLM[: 3 x
DRC[: 3 x
DWC[: 3 x
FKL[: 3 x
G>L[: 3 x
JDH[: 3 x
JKL[: 3 x
JVB[: 3 x
KBD[: 3 x
KRT[: 

Collect the "complements" of the lemor clauses.

In [35]:
lemor_daughters = {}
for l in sorted(lemor):
    daughters = Ci.mother.v(l)
    lemor_daughters[l] = daughters
msg("{} lemors have daughters".format(len(lemor_daughters)))

15m 49s 0 lemors have daughters
